# Product RAG Pipeline (MongoDB + Pinecone + MiniLM + FLAN-T5)
This notebook loads and cleans products.csv, inserts into MongoDB, builds MiniLM embeddings and upserts to Pinecone, then performs RAG search with price filters and generates a natural language answer using FLAN-T5.

In [1]:
%pip install -q pandas pymongo python-dotenv certifi sentence-transformers pinecone transformers torch tqdm

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os, re, json
from dotenv import load_dotenv
import certifi
from pymongo import MongoClient, ASCENDING, errors
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

load_dotenv()

MONGO_URI = os.getenv('ATLAS_URI')
if not MONGO_URI:
    raise RuntimeError('Missing ATLAS_URI in environment (.env)')
mongo_client = MongoClient(MONGO_URI, tlsCAFile=certifi.where())
db = mongo_client['ecommerce_db']
products_coll = db['products']

try:
    products_coll.create_index([('unique_id', ASCENDING)], unique=True, background=True)
    products_coll.create_index([('category', ASCENDING)], background=True)
    products_coll.create_index([('price', ASCENDING)], background=True)
except errors.PyMongoError as e:
    print('Index setup warning:', e)

embed_model = SentenceTransformer('all-MiniLM-L6-v2')  

PINECONE_API_KEY = os.getenv('Pinecone_API_KEY')
if not PINECONE_API_KEY:
    raise RuntimeError('Missing Pinecone_API_KEY in environment (.env)')
pc = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = 'prod-search'
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(name=INDEX_NAME, dimension=384, metric='cosine', spec=ServerlessSpec(cloud='aws', region='us-east-1'))
pinecone_index = pc.Index(INDEX_NAME)

print('Setup complete.')

c:\Users\Nikhil\Downloads\New folder\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete.


In [ ]:
import pandas as pd

def clean_price(value):
    if pd.isna(value):
        return 0.0
    s = str(value)
    s = re.sub(r'[^\d.,]', '', s).replace(',', '')
    try:
        return float(s) if s else 0.0
    except Exception:
        return 0.0

def clean_and_insert(csv_path='prod.csv'):
    df = pd.read_csv(csv_path)
    rename = {
        'Uniq Id': 'unique_id',
        'Product Title': 'title',
        'Product Description': 'description',
        'Brand': 'brand',
        'Price': 'price',
        'Image Urls': 'image_urls',
        'Category': 'category',
    }
    df = df.rename(columns=rename)

    df['title'] = df['title'].astype(str).str.strip()
    df['description'] = df.get('description', '').fillna('').astype(str).str.strip()
    df['brand'] = df.get('brand', '').astype(str).str.strip()
    df['category'] = df.get('category', '').astype(str).str.strip()
    df['image_urls'] = df.get('image_urls', '').fillna('')
    df['price'] = df['price'].apply(clean_price).astype(float)

    df = df.dropna(subset=['unique_id'])
    df['unique_id'] = df['unique_id'].astype(str)
    df = df.drop_duplicates(subset=['unique_id'])

    df['image_urls'] = df['image_urls'].apply(lambda x: str(x).split('|') if str(x) else [])

    upserts = 0
    for _, row in tqdm(df.iterrows(), total=len(df), desc='Upserting to Mongo'):
        doc = {
            'unique_id': row['unique_id'],
            'title': row['title'],
            'description': row['description'],
            'brand': row['brand'],
            'price': float(row['price']),
            'category': row['category'],
            'image_urls': row['image_urls'],
        }
        res = products_coll.update_one({'unique_id': row['unique_id']}, {'$set': doc}, upsert=True)
        if res.upserted_id is not None or res.modified_count > 0:
            upserts += 1
    print(f'✅ Upserted/updated {upserts} documents.')

clean_and_insert('prod.csv')

Upserting to Mongo: 100%|██████████| 5196/5196 [02:53<00:00, 29.90it/s]

✅ Upserted/updated 5196 documents.


In [ ]:
def upsert_embeddings_to_pinecone(batch_size=100):
    cursor = products_coll.find({}, {'unique_id': 1, 'title': 1, 'description': 1, 'price': 1, 'brand': 1})
    batch, total = [], 0
    for doc in tqdm(cursor, desc='Embedding + upsert'):
        uid = str(doc['unique_id'])
        text = f"{doc.get('title','')} [SEP] {doc.get('description','')}".strip()
        emb = embed_model.encode(text).tolist()
        meta = {
            'price': float(doc.get('price', 0.0)),
            'brand': str(doc.get('brand', ''))
        }
        batch.append({'id': uid, 'values': emb, 'metadata': meta})
        if len(batch) >= batch_size:
            pinecone_index.upsert(batch)
            total += len(batch)
            batch = []
    if batch:
        pinecone_index.upsert(batch)
        total += len(batch)
    print(f'✅ Upserted {total} vectors into Pinecone.')

upsert_embeddings_to_pinecone()

Embedding + upsert: 5196it [09:36,  9.02it/s]


✅ Upserted 5196 vectors into Pinecone.


In [2]:
def parse_price_filter(query: str):
    """
    Parse price filters from query. Supports:
    - Lower bounds: 'under 100', 'below 100', 'less than 100', 'under100'
    - Upper bounds: 'above 500', 'over 500', 'more than 500', 'above500'
    - Range: 'between 100 and 500', '100 to 500'
    Returns: dict with 'min_price' and/or 'max_price' keys
    """
    price_filter = {}
    
    lower_patterns = [
        r'(under|below|less\s*than)\s*[₹$]?\s*(\d+)',
        r'under\s*(\d+)',
        r'below\s*(\d+)'
    ]
    
    for pattern in lower_patterns:
        m = re.search(pattern, query, flags=re.I)
        if m:
            try:
                price = float([g for g in m.groups() if g and g.isdigit()][-1])
                price_filter['max_price'] = price
                break
            except (ValueError, IndexError):
                continue
    
    upper_patterns = [
        r'(above|over|more\s*than|greater\s*than)\s*[₹$]?\s*(\d+)',
        r'above\s*(\d+)',
        r'over\s*(\d+)',
        r'minimum\s*[₹$]?\s*(\d+)',
        r'at\s*least\s*[₹$]?\s*(\d+)'
    ]
    
    for pattern in upper_patterns:
        m = re.search(pattern, query, flags=re.I)
        if m:
            try:
                price = float([g for g in m.groups() if g and g.isdigit()][-1])
                price_filter['min_price'] = price
                break
            except (ValueError, IndexError):
                continue
    
    range_patterns = [
        r'between\s*[₹$]?\s*(\d+)\s*(?:and|to|-)\s*[₹$]?\s*(\d+)',
        r'(\d+)\s*(?:to|-)\s*(\d+)',
        r'from\s*[₹$]?\s*(\d+)\s*(?:to|-)\s*[₹$]?\s*(\d+)'
    ]
    
    for pattern in range_patterns:
        m = re.search(pattern, query, flags=re.I)
        if m:
            try:
                min_price = float(m.group(1))
                max_price = float(m.group(2))
                if min_price <= max_price:
                    price_filter['min_price'] = min_price
                    price_filter['max_price'] = max_price
                else:
                    price_filter['min_price'] = max_price
                    price_filter['max_price'] = min_price
                break
            except (ValueError, IndexError):
                continue
    
    return price_filter if price_filter else None

def search_products(query: str, top_k: int = 10):
    q_emb = embed_model.encode(query).tolist()
    res = pinecone_index.query(vector=q_emb, top_k=top_k, include_metadata=True)
    matches = res.get('matches', [])
    if not matches:
        return {'status': 'not_found', 'message': 'No vector matches.'}

    ids = [m['id'] for m in matches]
    price_filter = parse_price_filter(query)

    mongo_filter = {'unique_id': {'$in': ids}}
    if price_filter:
        price_query = {}
        if 'min_price' in price_filter:
            price_query['$gte'] = price_filter['min_price']
        if 'max_price' in price_filter:
            price_query['$lte'] = price_filter['max_price']
        if price_query:
            mongo_filter['price'] = price_query
    
    docs = list(products_coll.find(mongo_filter))
    if not docs:
        filter_msg = ""
        if price_filter:
            if 'min_price' in price_filter and 'max_price' in price_filter:
                filter_msg = f" with price between ₹{price_filter['min_price']} and ₹{price_filter['max_price']}"
            elif 'min_price' in price_filter:
                filter_msg = f" with price above ₹{price_filter['min_price']}"
            elif 'max_price' in price_filter:
                filter_msg = f" with price under ₹{price_filter['max_price']}"
        return {'status': 'not_found', 'message': f'No products matched after filtering{filter_msg}.'}

    score_map = {m['id']: m.get('score', 0) for m in matches}
    docs.sort(key=lambda d: score_map.get(str(d.get('unique_id')), 0), reverse=True)
    
    result = {'status': 'success', 'results': docs}
    if price_filter:
        result['price_filter'] = price_filter
    
    return result

In [3]:
from transformers import pipeline

gen = pipeline('text2text-generation', model='google/flan-t5-small')

def generate_response(query: str, products: list):
    if not products:
        return 'I could not find matching products right now.'
    
    top = products[:3] 
    brands = list({str(p.get('brand','')).strip() for p in top if p.get('brand') and str(p.get('brand','')).strip()})
    pr = [float(p.get('price', 0)) for p in top if isinstance(p.get('price', 0), (int, float)) and p.get('price', 0) > 0]
    pr_min = min(pr) if pr else None
    pr_max = max(pr) if pr else None
    
    product_info = []
    for i, p in enumerate(top, 1):
        title = str(p.get('title', 'Product')).strip()
        if len(title) > 50:
            title = title[:47] + "..."
        brand = str(p.get('brand', 'Unknown')).strip()
        price = p.get('price', 0)
        product_info.append(f"{i}. {title} by {brand} (₹{price})")
    
    brand_text = f"from brands like {', '.join(brands[:2])}" if brands else ""
    price_text = ""
    if pr_min and pr_max:
        if pr_min == pr_max:
            price_text = f"priced at ₹{pr_min}"
        else:
            price_text = f"ranging from ₹{pr_min} to ₹{pr_max}"
    
    prompt = f"""Write a helpful product recommendation response.And try to convince the user
Write 2-3 sentences explaining why these are good matches {brand_text} {price_text}."""

    try:
        result = gen(
            prompt, 
            max_new_tokens=80, 
            do_sample=True,
            num_beams=3,
            early_stopping=True,
            pad_token_id=gen.tokenizer.eos_token_id
        )
        
        response = result[0]['generated_text'].strip()
        
        if response:
            sentences = response.split('.')
            unique_sentences = []
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence not in unique_sentences:
                    unique_sentences.append(sentence)
            response = '. '.join(unique_sentences[:3]) 
            if response and not response.endswith('.'):
                response += '.'
        
        return response if response else f"I found {len(products)} relevant products for your search."
        
    except Exception as e:
        print(f"Generation error: {e}")
        brand_mention = f" from {brands[0]}" if brands else ""
        price_mention = f" under ₹{pr_max}" if pr_max else ""
        return f"I found {len(products)} great products matching your search{brand_mention}{price_mention}. These options should meet your needs perfectly."

Device set to use cuda:0


In [7]:

test_queries = [
    'premium shampoo above 500',
    'moisturizer between 200 and 800',
    'organic products over 300',
    'longlasting french aroma perfume'
]

for query in test_queries:
    print(f"\nTesting: '{query}'")
    print("=" * 60)
    
    res = search_products(query, top_k=10)
    if res.get('status') == 'success':
        prods = res['results']
        price_filter = res.get('price_filter', {})
        
        print(f'Found {len(prods)} products')
        if price_filter:
            filter_desc = []
            if 'min_price' in price_filter:
                filter_desc.append(f"min: ₹{price_filter['min_price']}")
            if 'max_price' in price_filter:
                filter_desc.append(f"max: ₹{price_filter['max_price']}")
            print(f'Price filter applied: {", ".join(filter_desc)}')

        print('\nTop 3 results:')
        for i, p in enumerate(prods[:3], 1):
            print(f'{i}. {p.get("title", "N/A")}')
            print(f'   Brand: {p.get("brand", "N/A")} | Price: ₹{p.get("price", 0)}')

        print('\nResponse :')
        print(generate_response(query,prods))
    else:
        print(f'{res.get("message", "No results found")}')
print()


Testing: 'premium shampoo above 500'
Found 4 products
Price filter applied: min: ₹500.0

Top 3 results:
1. Scentio Hair Professional Argan Oil Therapy Shampoo, 500 ml
   Brand: SCENTIO | Price: ₹819.0
2. Deptrol Car Wash Shampoo 500 ml Pack of 24
   Brand: Deptrol | Price: ₹3146.0
3. Peter Thomas Roth Mega-Rich Shampoo 8.5 fl oz.
   Brand: Peter | Price: ₹3344.0

Response :
SCENTIO is a brand that has been rated PG-13. It is a good match for SCENTIO. It is rated PG-13.

Testing: 'moisturizer between 200 and 800'
Found 1 products
Price filter applied: min: ₹200.0, max: ₹800.0

Top 3 results:
1. Jergens Dry Skin Moisturiser (600ml)
   Brand: Jergens | Price: ₹715.0

Response :
135715.

Testing: 'organic products over 300'


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Found 2 products
Price filter applied: min: ₹300.0

Top 3 results:
1. Adi Naturals Organic Tamarind - 500 GMS Pack of 3
   Brand: Adi Naturals | Price: ₹390.0
2. Tates The Natural Miracle Conditioner 100% Organic 16 Oz.
   Brand: Tates | Price: ₹3240.0

Response :
It's a bit pricey, but it's worth the money.

Testing: 'longlasting french aroma perfume'
Found 10 products

Top 3 results:
1. Remy Marquis (Paris) Pour Femme Eau De Parfum, 100ml
   Brand: Remy Marquis | Price: ₹1110.0
2. Adf - Anais De France Floral Beauty Perfume For Women
   Brand: ADF - ANAIS DE FRANCE | Price: ₹210.0
3. Verser Golden Oudh Attar Pure Natural Roll on Long Lasting Fragrance Perfume for Men & Women (Non-alcoholic)
   Brand: Verser | Price: ₹2800.0

Response :
ANAIS DE FRANCE is a brand that has a high percentage of sales.

